### Import libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# from nltk.corpus import stopwords
from nltk.util import ngrams

from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix

from collections import defaultdict
from collections import Counter
plt.style.use('ggplot')
import nltk
nltk.download('stopwords')
# stop=set(stopwords.words('english'))

import re
from nltk.tokenize import word_tokenize
import gensim
import string

from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam

import warnings
warnings.simplefilter('ignore')

import nltk
stop = set(nltk.corpus.stopwords.words('english'))
print(stop)

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Get Data

In [ ]:
tweet= pd.read_csv('./Data/train.csv')
test=pd.read_csv('./Data/test.csv')
submission = pd.read_csv("./Data/sample_submission.csv")

In [ ]:
# Correct wrong data
# From https://www.kaggle.com/wrrosa/keras-bert-using-tfhub-modified-train-data - 
# author of this kernel read tweets in training data and figure out that some of them have errors:
ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
tweet.loc[tweet['id'].isin(ids_with_target_error),'target'] = 0
tweet[tweet['id'].isin(ids_with_target_error)]

In [ ]:
print('There are {} rows and {} columns in train'.format(tweet.shape[0],tweet.shape[1]))
print('There are {} rows and {} columns in test'.format(test.shape[0],test.shape[1]))

In [ ]:
tweet.head(10)

In [ ]:
df=pd.concat([tweet,test])
tweet.shape
test.shape
df.shape

## Useful Preprocessing/Data cleaning

#### Removing URLs

In [ ]:
example="New competition launched :https://www.kaggle.com/c/nlp-getting-started"

In [ ]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

remove_URL(example)

In [ ]:
df['text']=df['text'].apply(lambda x : remove_URL(x))

#### Removing HTML tags

In [ ]:
example = """<div>
<h1>Real or Fake</h1>
<p>Kaggle </p>
<a href="https://www.kaggle.com/c/nlp-getting-started">getting started</a>
</div>"""

In [ ]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
print(remove_html(example))

In [ ]:
df['text']=df['text'].apply(lambda x : remove_html(x))

#### Not removing but handling emoji and emoticons

In [ ]:
# installation and importing the library
!pip install emot
import emot

In [ ]:
def replace_emojis(text):
    emoji_details=emot.emoji(text)
    meaning=re.compile(r':')
    emojis_meaning=""
    for m in emoji_details['mean']:
        emojis_meaning=emojis_meaning+" "+meaning.sub(r'',m)
    return emojis_meaning

In [ ]:
def replace_emoticons(text):
    emoticon_details=emot.emoticons(text)
    emoticon_meaning=""
    print(emoticon_details['mean'])
    for m in emoticon_details['mean']:
        emoticon_meaning=emoticon_meaning+" "+ str(m)
    return emoticon_meaning

In [ ]:
example_df= pd.DataFrame(["The weather is ☁️ :), we might need to carry our ☂️ :("])
example_df.columns=['text']
example_df['emojis_meaning']=example_df['text'].apply(lambda x: replace_emojis(x))
example_df['emoticon_meaning']=example_df['text'].apply(lambda x: replace_emoticons(x))
print(example_df)

In [ ]:
df['emojis_meaning']=df['text'].apply(lambda x: replace_emojis(x))
# df['emoticon_meaning']=df['text'].apply(lambda y: replace_emoticons(y))

In [ ]:
print(df[0:5])

In [ ]:
df['emojis_meaning'].describe(include="all")

In [ ]:
df['emojis_meaning'].unique

#### Removing emoji and emoticons, in case handling is not possible

In [ ]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

remove_emoji("Omg another Earthquake 😔😔")

In [ ]:
df['text']=df['text'].apply(lambda x: remove_emoji(x))

#### Removing punctuations

In [ ]:
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

example="I am a #king"
print(remove_punct(example))

In [ ]:
df['text']=df['text'].apply(lambda x : remove_punct(x))

In [ ]:
df.describe()

In [ ]:
df

In [ ]:
training_df=df[df['target'].notnull()]

In [ ]:
oot_df=df[df['target'].isnull()]

In [ ]:
training_df.shape
oot_df.shape

In [ ]:
oot_df.describe(include="all")

In [ ]:
oot_df = oot_df.drop('target', 1)

### Trying simple model now

In [ ]:
def cv(data):
    count_vectorizer = CountVectorizer()

    emb = count_vectorizer.fit_transform(data)

    return emb, count_vectorizer

list_corpus = training_df["text"].tolist()
list_labels = training_df["target"].tolist()
oot_corpus = oot_df["text"].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
                                                                                random_state=32)

X_train_counts, count_vectorizer = cv(X_train)
X_test_counts = count_vectorizer.transform(X_test)
oot_df_counts = count_vectorizer.transform(oot_corpus)

In [ ]:
len(X_train)
# len(X_train_counts)

In [ ]:

## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
clf = linear_model.RidgeClassifier()

In [ ]:
scores = model_selection.cross_val_score(clf, X_train_counts, y_train, cv=3, scoring="f1")
scores

In [ ]:
scores = model_selection.cross_val_score(clf, X_test_counts, y_test, cv=3, scoring="f1")
scores

In [ ]:
clf.fit(X_train_counts, y_train)

In [ ]:
sample_submission = pd.read_csv("./Data/sample_submission.csv")

In [ ]:
sample_submission["target"] = clf.predict(oot_df_counts)

In [ ]:
sample_submission.describe()

In [ ]:
#why am i getting score of 0?

In [ ]:
sample_submission.to_csv("submission.csv", index=False)

### Trying simple model on whole dataset

In [ ]:
def cv(data):
    count_vectorizer = CountVectorizer()

    emb = count_vectorizer.fit_transform(data)

    return emb, count_vectorizer

list_corpus = training_df["text"].tolist()
list_labels = training_df["target"].tolist()
oot_corpus = oot_df["text"].tolist()

# X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
#                                                                                 random_state=32)

X_train_counts, count_vectorizer = cv(list_corpus)
# X_test_counts = count_vectorizer.transform(X_test)
oot_df_counts = count_vectorizer.transform(oot_corpus)

In [ ]:
clf = linear_model.RidgeClassifier()

In [ ]:
clf.fit(X_train_counts, list_labels)

In [ ]:
scores = model_selection.cross_val_score(clf, X_train_counts, list_labels , cv=3, scoring="f1")
scores

In [ ]:
sample_submission = pd.read_csv("./Data/sample_submission.csv")
sample_submission["target"] = clf.predict(oot_df_counts)
sample_submission.describe()
sample_submission.to_csv("submission.csv", index=False)

### TF-IDF vectorizer

In [ ]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()

    train = tfidf_vectorizer.fit_transform(data)

    return train, tfidf_vectorizer

X_train_tfidf, tfidf_vectorizer = tfidf(list_corpus)
X_oot_tfidf = tfidf_vectorizer.transform(oot_corpus)


In [ ]:
clf = linear_model.RidgeClassifier()

In [ ]:
print(len(list_labels))

In [ ]:
clf.fit(X_train_tfidf, list_labels)

In [ ]:
scores = model_selection.cross_val_score(clf, X_train_tfidf, list_labels , cv=3, scoring="f1")
scores

In [ ]:
sample_submission = pd.read_csv("./Data/sample_submission.csv")
sample_submission["target"] = clf.predict(X_oot_tfidf)
sample_submission.describe()
sample_submission.to_csv("submission.csv", index=False)